In [17]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

In [2]:
amp1_list = #file names before "_R*"
amp2_list = #file names before "_R*"
samples = # sample names
path = # path containing repertoires

In [3]:
amp1_list = np.array(amp1_list)[[1,2]]
amp2_list = np.array(amp2_list)[[1,2]]
samples = np.array(samples)[[1,2]]
amp1_list, amp2_list, samples = list(amp1_list), list(amp2_list), list(samples)

IndexError: index 1 is out of bounds for axis 0 with size 1

In [44]:
for amp1, amp2, sample in tqdm_notebook(zip(amp1_list, amp2_list, samples), total=len(samples)):
    
    amp1H_file = pd.read_csv(path + amp1 + "_2.clones_IGH.tsv", sep="\t")
    
    amp2H_file = pd.read_csv(path + amp2 + "_IGH.alignments.tsv", sep="\t")
    amp2Hc_file = pd.read_csv(path + amp2 + "_IGH.clones_IGH.tsv", sep="\t")
    amp2KL_file = pd.read_csv(path + amp2 + "_IGKL.alignments.tsv", sep="\t")

    amp1H_file.drop(columns=["bestCHit", "readCount", "readFraction", "cloneId"], inplace=True)
    amp2H_file = amp2H_file[["cloneId", "readId","nSeqCDR3", "bestJHit"]] # without V/D matches
    amp2Hc_file = amp2Hc_file[["cloneId", "readCount", "readFraction"]]
    amp2H_file = amp2H_file.merge(amp2Hc_file, on=["cloneId"])
    
    H_file = amp1H_file.merge(amp2H_file, on=["nSeqCDR3", "bestJHit"])
    H_file = H_file.add_prefix("IGH_")

    amp2KL_file = amp2KL_file.loc[amp2KL_file["cloneId"] != -1, ["cloneId", "readId", "nSeqCDR3"]]
    
    amp2KL_file_c = pd.concat([pd.read_csv(path + amp2 + "_IGKL.clones_IGK.tsv", sep="\t"),
                              pd.read_csv(path + amp2 + "_IGKL.clones_IGL.tsv", sep="\t")])

    amp2KL_file_c.drop(columns=["bestDHit", "bestCHit"], inplace=True)
    KL_file = amp2KL_file.merge(amp2KL_file_c, on=["cloneId", "nSeqCDR3"])
    KL_file = KL_file.add_prefix("IGKL_")

    
    
    df_total = H_file.merge(KL_file, left_on="IGH_readId", right_on="IGKL_readId")
    print(len(df_total["IGH_readId"].unique()))
    df_total.drop(columns=["IGH_readId", "IGKL_readId"], inplace=True)

    good_cols = ["cloneCount", "bestVHit", "bestDHit", "bestJHit", "bestCHit",
                 "nSeqCDR3", "cloneId", "cloneFraction", "targetSequences", "aaSeqCDR3",
                 "readCount","readFraction"]
    IGH_cols = [_ for _ in list(df_total.columns) if "IGH" in _ and _.split("_")[1] in good_cols]
    IGKL_cols = [_ for _ in list(df_total.columns) if "IGKL" in _ and _.split("_")[1] in good_cols]
    
    
    df_total = df_total.groupby(IGH_cols + IGKL_cols).size().reset_index(name='match')
    df_total['IGKL_chains'] = df_total['IGKL_bestJHit'].str[:3]
    
    # Renaming "read" to "clone" prefix to fit the previous format
    df_total.rename(columns={"IGH_readCount":"IGH_cloneCount", "IGKL_readCount":"IGKL_cloneCount",
                            "IGH_readFraction":"IGH_cloneFraction", "IGKL_readFraction":"IGKL_cloneFraction"},
                    inplace=True)
    
    # Merging by the identical CDR3aa
    cols = [col for col in df_total.columns if any(s in col for s in ["cloneId", "best", "nSeqCDR3", "chains", "targetSequences"])]
    df_total = df_total.groupby(["IGH_aaSeqCDR3","IGKL_aaSeqCDR3"]).agg({"IGH_cloneCount":"sum", "IGKL_cloneCount":"sum",
                                                         "IGH_cloneFraction":"sum", "IGKL_cloneFraction":"sum",
                                                         "match":"sum",
                                                         "IGH_cloneId":"unique", "IGKL_cloneId":"unique",
                                                         "IGKL_chains":"unique",
                                                         "IGH_bestVHit":"unique", "IGH_bestDHit":"unique", "IGH_bestJHit":"unique",
                                                         "IGKL_bestVHit":"unique", "IGKL_bestJHit":"unique",
                                                         "IGH_nSeqCDR3":"unique", "IGH_targetSequences":"unique",
                                                         "IGKL_nSeqCDR3":"unique", "IGKL_targetSequences":"unique"
                                                         })
    for col in cols:
        df_total[col] = [";".join(df_total[col][i].astype(str)) for i in range(len(df_total))]
    df_total = df_total.reset_index()
    
    # Filter clones with a low count
    df_total = df_total.loc[(df_total["IGH_cloneCount"] > 2) & (df_total["IGKL_cloneCount"] > 2)]
    
    
    df_total["IGH_matchFrac"] = df_total["match"] / df_total["IGH_cloneCount"]
    df_total["IGKL_matchFrac"] = df_total["match"] / df_total["IGKL_cloneCount"]
    
    df_total = df_total[["IGH_cloneId", "IGH_cloneCount", "IGH_cloneFraction", "IGH_bestVHit", "IGH_bestDHit", "IGH_bestJHit", "IGH_nSeqCDR3", "IGH_aaSeqCDR3", "IGH_targetSequences",
                         "IGKL_cloneId", "IGKL_chains", "IGKL_cloneCount", "IGKL_cloneFraction", "IGKL_bestVHit", "IGKL_bestJHit", "IGKL_nSeqCDR3", "IGKL_aaSeqCDR3", "IGKL_targetSequences", "IGH_matchFrac",  "IGKL_matchFrac", "match"]]

    df_total.rename(columns={"IGKL_cloneCount":"IGKL_count",
                            "IGH_cloneFraction":"IGH_freq",
                             "IGKL_cloneFraction":"IGKL_freq"}, inplace=True)
    
    df_total["IGKL_sym"] = df_total["IGH_freq"] / (df_total["IGKL_freq"] * df_total["IGKL_matchFrac"] )
    df_total["IGH_sym"] = df_total["IGH_freq"] * df_total["IGH_matchFrac"] / df_total["IGKL_freq"]
    
    df_total["IGKL_symNorm"] = df_total["IGKL_sym"].apply(lambda x: 100**x if x <= 1 else 100**(1/x))
    df_total["IGH_symNorm"] = df_total["IGH_sym"].apply(lambda x: 100**x if x <= 1 else 100**(1/x))
    
    df_total["signal"] = df_total["IGKL_matchFrac"] / df_total["IGH_freq"]
    
    df_total["symNorm_HMean"] = 2*df_total["IGKL_symNorm"]*df_total["IGH_symNorm"]/(df_total["IGKL_symNorm"] + df_total["IGH_symNorm"])
    
    df_total = df_total.sort_values(by=["IGH_cloneId","IGKL_cloneId"]).reset_index(drop=True)
    df_total.to_csv(path + sample + '_IGH-fusion_result.collapsed.txt', sep="\t", index=False)

/scratch/ipykernel_2353969/1946420187.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for amp1, amp2, sample in tqdm_notebook(zip(amp1_list, amp2_list, samples), total=len(samples)):


  0%|          | 0/1 [00:00<?, ?it/s]

/scratch/ipykernel_2353969/1946420187.py:5: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  amp2H_file = pd.read_csv(path + amp2 + "_IGH.alignments.tsv", sep="\t")
/scratch/ipykernel_2353969/1946420187.py:7: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  amp2KL_file = pd.read_csv(path + amp2 + "_IGKL.alignments.tsv", sep="\t")


38408


In [43]:
df_total

,IGH_cloneId,IGH_cloneCount,IGH_freq,IGH_bestVHit,IGH_bestDHit,IGH_bestJHit,IGH_nSeqCDR3,IGH_aaSeqCDR3,IGH_targetSequences,IGKL_cloneId,...,IGKL_targetSequences,IGH_matchFrac,IGKL_matchFrac,match,IGKL_sym,IGH_sym,IGKL_symNorm,IGH_symNorm,signal,symNorm_HMean
0,0,35029.0,0.801432,IGHV3-33*00,IGHD6-6*00,IGHJ4*00,TGTGCGACCAGAATCGAGGATAGCAGCTCGGCTGCCGGAGGGTGGG...,CATRIEDSSSAAGGWELDYW,TGTGCGACCAGAATCGAGGATAGCAGCTCGGCTGCCGGAGGGTGGG...,0,...,TGTCAACAGAGTTACAGTACCTCGTGGACGTTC,0.000057,0.000047,2,19488.026872,0.000052,1.000236,1.000242,0.000059,1.000239
1,0,35029.0,0.801432,IGHV3-33*00,IGHD6-6*00,IGHJ4*00,TGTGCGACCAGAATCGAGTATAGCAGCGCGGCTGCCGGAGGGTGGG...,CATRIEYSSAAAGGWELDYW,TGTGCGACCAGAATCGAGTATAGCAGCGCGGCTGCCGGAGGGTGGG...,0,...,TGTCAACAGAGTTACAGTACCTCGTGGACGTTC,0.000714,0.000590,25,1559.042150,0.000656,1.002958,1.003026,0.000736,1.002992
2,0,35029.0,0.801432,IGHV3-33*00,IGHD6-6*00,IGHJ4*00,TGTGCGACCAGAATCGAGTATAGCAGCTCGGCTGACGGAGGGTGGG...,CATRIEYSSSADGGWELDYW,TGTGCGACCAGAATCGAGTATAGCAGCTCGGCTGACGGAGGGTGGG...,0,...,TGTCAACAGAGTTACAGTACCTCGTGGACGTTC,0.001342,0.001109,47,829.277739,0.001234,1.005569,1.005697,0.001383,1.005633
3,0,35029.0,0.801432,IGHV3-33*00,IGHD6-6*00,IGHJ4*00,GGTGCGACCAGAATCGAGTATAGCAGCTCGGCTGCCGGAGGGTGGG...,GATRIEYSSSAAGGWELDYW,GGTGCGACCAGAATCGAGTATAGCAGCTCGGCTGCCGGAGGGTGGG...,0,...,TGTCAACAGAGTTACAGTACCTCGTGGACGTTC,0.000057,0.000047,2,19488.026872,0.000052,1.000236,1.000242,0.000059,1.000239
4,0,35029.0,0.801432,IGHV3-33*00,IGHD6-6*00,IGHJ4*00,TGGGCGACCAGAATCGAGTATAGCAGCTCGGCTGCCGGAGGGTGGG...,WATRIEYSSSAAGGWELDYW,TGGGCGACCAGAATCGAGTATAGCAGCTCGGCTGCCGGAGGGTGGG...,0,...,TGTCAACAGAGTTACAGTACCTCGTGGACGTTC,0.000057,0.000047,2,19488.026872,0.000052,1.000236,1.000242,0.000059,1.000239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,9,330.0,0.007550,IGHV3-33*00,IGHD5-18*00,IGHJ4*00,TGTGCGAGAGATAAGACAGCTAGTCTCGGCTACTGG,CARDKTASLGYW,TGTGCGAGAGATAAGACAGCTAGTCTCGGCTACTGG,0,...,TGTCAACAGAGTTACAGTACCTCGTGGACGTTC,0.148485,0.001156,49,7.493556,0.001286,1.848826,1.005940,0.153091,1.302950
161,9,330.0,0.007550,IGHV3-33*00,IGHD5-18*00,IGHJ4*00,TGTGCGAGAGATAAGACAGCTAGTCTCGGCTACTGG,CARDKTASLGYW,TGTGCGAGAGATAAGACAGCTAGTCTCGGCTACTGG,2,...,TGCATGCAAGCTCTACAAACTCCGAGGACGTTC,0.921212,0.614141,304,1.207843,0.683343,45.273530,23.264046,81.342100,30.734833
162,9,330.0,0.007550,IGHV3-33*00,IGHD5-18*00,IGHJ4*00,TGTGCGAGAGATAAGACAGCTAGTCTCGGCTACTGG,CARDKTASLGYW,TGTGCGAGAGATAAGACAGCTAGTCTCGGCTACTGG,20,...,TGTCAGCAGTATGGTAGCTCACCTCCGTTC,0.003030,0.009524,1,367.184268,0.010597,1.012621,1.050011,1.261414,1.030977
163,9,330.0,0.007550,IGHV3-33*00,IGHD5-18*00,IGHJ4*00,TGTGCGAGAGATAAGACAGCTAGTCTCGGCTACTGG,CARDKTASLGYW,TGTGCGAGAGATAAGACAGCTAGTCTCGGCTACTGG,28,...,TGTCAACACTATTATATTGCTCCATTCACTTTC,0.003030,0.033333,1,367.184268,0.037089,1.012621,1.186257,4.414949,1.092583
